# Intelligent Scene

Intelligent Scene creates scenes for a zone in a Philips Hue light setup using artificial intelligence.

It is powered by the Microsoft Semantic Kernel and uses Open AI Chat Completion to create the scene based on:
* The location of the zone, e.g. Ground Floor,
* The number of lights the zone contains, e.g. 4,
* A situation such an event or environment for the scene, e.g. Woodland on a Sunny Day, a Birthday Party, etc..

**This is a notebook version of the initial implementation as seen in my [IoT Experiments](https://github.com/bhazel/iot-experiments) repository on GitHub.**

## Initial Set Up

**You need to have a zone configured in your Philips Hue installation for this notebook to run correctly.**

Intelligent Scene uses the [Microsoft Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/overview/) to orchestrate the AI and the [Q42.HueApi](https://github.com/michielpost/Q42.HueApi) to interact with Philips Hue.  These are both installed from NuGet.

The necessary `using` statements provide access to their APIs.  Additional types for the created scene are imported for a utility file.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"
#r "nuget: HueApi, 1.6.0"

In [ ]:
using System;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Text.Json.Nodes;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.SemanticKernel;
using HueApi;
using HueApi.Models;
using HueApi.Models.Requests;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using static System.Console;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

In [ ]:
#!import ./Utils/Model.cs

## Load Configuration

All values used by Intelligent Scene are set in the `config.json` file in the same directory as this notebook.

* **Philips Hue:**
    * **Bridge IP Address:** The IP address of the Hue Bridge, e.g. `192.168.1.50`.
    * **App Username:** The Hue Bridge app username, e.g. `ABCDEFGHIJKLMNOPQRSTUVWXYZABC-abcdefghij`.
    * **Zone:** The zone to set the scene, e.g. `Ground Floor`.
* **Open AI:**
    * **API Key:** The Open AI API key, e.g. `sk-ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuv`.
    * **Model:** The Open AI model to use, e.g. `gpt-3.5-turbo`.

Any values which are not provided are either requested via prompts or automatically.

In [ ]:
string configurationText = File.ReadAllText("config.json");
JsonNode configuration = JsonNode.Parse(configurationText);
string philipsHueBridgeIpAddress = configuration["PhilipsHue"]["BridgeIpAddress"]!.ToString();
string philipsHueAppUsername = configuration["PhilipsHue"]["AppUsername"]!.ToString();

display(span(
    strong("Philips Hue Bridge IP Address: "), !string.IsNullOrWhiteSpace(philipsHueBridgeIpAddress) ? philipsHueBridgeIpAddress : em("Not Provided"),
    br(),
    strong("Philips Hue App Username: "), !string.IsNullOrWhiteSpace(philipsHueAppUsername) ? em("Provided") : em("Not Provided")
));

In [ ]:
if (string.IsNullOrWhiteSpace(philipsHueBridgeIpAddress))
{
    HttpClient httpClient = new HttpClient();
    string ipDiscoveryResponse = await httpClient.GetStringAsync("https://discovery.meethue.com");
    JsonNode ipDiscovery = JsonNode.Parse(ipDiscoveryResponse);
    philipsHueBridgeIpAddress = ipDiscovery[0]["internalipaddress"]!.ToString();
    display(span(
        strong("Philips Hue Bridge IP Address (via Discovery): "),
        philipsHueBridgeIpAddress
    ));
}

if (string.IsNullOrWhiteSpace(philipsHueAppUsername))
{
    philipsHueAppUsername = await GetInputAsync("Please enter youe Philips Hue app username");
    display(span(
        strong("Philips Hue App Username (via Input): "),
        !string.IsNullOrWhiteSpace(philipsHueBridgeIpAddress) ? em("Provided") : em("Not Provided")
    ));
}

## Load Zones

Available zones in the current Philips Hue installation are loaded.  If one has not been provided in configuration you will be prompted for one here.

In [ ]:
LocalHueApi localHueApi = new(philipsHueBridgeIpAddress, philipsHueAppUsername);
List<Zone> zones = (await localHueApi.GetZonesAsync()).Data;
string[] zoneNames = zones.Select(zone => zone.Metadata!.Name).ToArray();
display(span(
    strong($"{zoneNames.Count()} Zone(s) Found: "),
    ul(
        zoneNames.Select(zone => li(zone))
    )
));

string philipsHueZone = configuration["PhilipsHue"]["Zone"]!.ToString();
display(span(
    strong("Philips Hue Zone: "), !string.IsNullOrWhiteSpace(philipsHueZone) ? philipsHueZone : em("Not Provided")
));

In [ ]:
if (string.IsNullOrWhiteSpace(philipsHueZone))
{
    philipsHueZone = await GetInputAsync($"Please select a zone (Available: {string.Join((", "), zoneNames)}):");
    display(span(
        strong("Philips Hue Zone (via Input): "),
        philipsHueZone
    ));
}

## Load Lights for Selected Zone

The available lights for the selected zone are loaded.

In [ ]:
Zone zone = zones.FirstOrDefault(z => z.Metadata!.Name == philipsHueZone);
List<Light> zoneLights = zone!.Children
    .Where(child => child.Rtype == "light")
    .Select(rid => localHueApi.GetLightAsync(rid.Rid))
    .Select(task => task.Result.Data.FirstOrDefault())
    .ToList();

display(span(
    strong($"{zoneLights.Count()} Light(s) Found in Zone '{philipsHueZone}':"),
    ul(
        zoneLights.Select(light => li(light!.Metadata!.Name))
    )
));

## Create & Configure Kernel

A kernel is created with Open AI chat completion integration using a specified model.  If either an Open AI API key or model is not provided you will be prompted for them here.

In [ ]:
string openAiApiKey = configuration["OpenAi"]["ApiKey"]!.ToString();
string openAiModel = configuration["OpenAi"]["Model"]!.ToString();

InteractiveKernel.display(span(
    strong("OpenAI API Key: "), !string.IsNullOrWhiteSpace(openAiApiKey) ? em("Provided") : em("Not Provided"),
    br(),
    strong("OpenAI Model: "), !string.IsNullOrWhiteSpace(openAiModel) ? openAiModel : em("Not Provided")
));

In [ ]:
if (string.IsNullOrWhiteSpace(openAiApiKey))
{
    openAiApiKey = await GetInputAsync("Please enter your OpenAI API key");
    display(span(
        strong("OpenAI API Key (via Input): "),
        !string.IsNullOrWhiteSpace(openAiApiKey) ? em("Provided") : em("Not Provided")
    ));
}

if (string.IsNullOrWhiteSpace(openAiModel))
{
    openAiModel = await GetInputAsync("Please enter your OpenAI model");
    display(span(
        strong("OpenAI Model (via Input): "),
        !string.IsNullOrWhiteSpace(openAiModel) ? openAiModel : em("Not Provided")
    ));
}

In [ ]:
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddOpenAIChatCompletion(openAiModel, openAiApiKey);
Kernel kernel = kernelBuilder.Build();

## Load Intelligent Scene Kernel Plug-in

Intelligent Scene is powered by a prompt-based plug-in which provides the instructions to the Open AI model on how to generate a scene.  The prompt can be seen in the `Plugins/IntelligentScene/skprompt.txt` file.

In [ ]:
string pluginsPath = Path.Combine(Environment.CurrentDirectory, "Plugins");
KernelPlugin kernelPlugins = kernel.ImportPluginFromPromptDirectory(pluginsPath);

display(span(
    strong($"{kernelPlugins.Count()} Plugin(s) Found:"),
    ul(
        kernelPlugins.Select(plugin => li(plugin.Name))
    )
));

## Prompt for a Situation & Submit to Open AI!

The prompt contains parameters which are configured in the kernel arguments and correspond to:
* **location:** The Philips Hue zone.
* **lights:** The number of lights in the zone.
* **situation:** The user-provided situation to create the scene.

You will be prompted for the situation in this cell.  This can be written in natural language and be as basic or detailed as required.  Examples include:
* Woodland on a sunny summer day in the shade of the trees.
* My birthday party for me and my friends.
* A romantic moodlit evening with my partner.
* Film night with an ocean theme.
* Working from home and I need to concentrate.
* Relaxed lighting with purple colours.

These are submitted to Open AI to create a scene.  The response is in JSON which is parsed and used to set the colours of the lights in the zone.

This cell contains an infinite loop to make another scene once one has completed.  To exit press the Escape key.

In [ ]:
while (true)
{
    string location = philipsHueZone;
    int lights = zoneLights.Count;
    string situation = await GetInputAsync("Please provide a situation for the scene.  Examples include: 'Woodland on a sunny summer day', 'A beatiful tropical sunset', 'Working from home and I need to focus', etc..");

    KernelArguments kernelArguments = new()
    {
        ["location"] = philipsHueZone,
        ["lights"] = lights,
        ["situation"] = situation
    };

    display(span(
        strong("Context for Prompt:"),
        ul(
            kernelArguments.Select(argument => li(
                strong($"{argument.Key}: "),
                argument.Value.ToString()
            ))
        )
    ));
    
    FunctionResult intelligentSceneResult = await kernel.InvokeAsync(kernelPlugins["IntelligentScene"], kernelArguments);
    string intelligentScene = intelligentSceneResult.ToString()
        .Replace("```json", string.Empty)
        .Replace("```", string.Empty)
        .Trim();
    
    display(span(
        strong("JSON Response:"),
        pre(intelligentScene)
    ));

    SceneInfo scene = JsonSerializer.Deserialize<SceneInfo>(intelligentScene);
    Dictionary<Light, ColourInfo> lightSettings = zoneLights
        .Zip(scene!.Colours, (light, colour) => new KeyValuePair<Light, ColourInfo>(light!, colour))
        .ToDictionary(lightColourPair => lightColourPair.Key, pair => pair.Value);

    foreach (KeyValuePair<Light, ColourInfo> lightSetting in lightSettings)
    {
        Light light = lightSetting.Key;
        ColourInfo colour = lightSetting.Value;
        UpdateLight updateLightRequest = new UpdateLight()
            .TurnOn()
            .SetBrightness(colour.Brightness)
            .SetColor(colour.Xy.X, colour.Xy.Y);
        
        await localHueApi.UpdateLightAsync(light.Id, updateLightRequest);
    }

    display(span(
        strong("Itelligent Scene Result:"),
        ul(
            lightSettings.Select(lightSetting => li(
                strong($"{lightSetting.Key.Metadata!.Name}: "),
                lightSetting.Value.Name
            ))
        )
    ));
}